# Limpieza de Todos los Datasets Iniciales

In [1]:
import pandas as pd
import numpy as np

In [2]:
from ydata_profiling import ProfileReport

/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [3]:
curdf = pd.read_pickle('../data/interim/curdf.pkl')
print('cur shape: ',curdf.shape)
datdf = pd.read_pickle('../data/interim/datdf_clean.pkl')
print('datdf shape: ',datdf.shape)
findf = pd.read_pickle('../data/interim/findf_01.pkl')
print('fin shape: ',findf.shape)

cur shape:  (581295, 12)
datdf shape:  (24830, 5)
fin shape:  (554523, 14)


## Alumnos a eliminar de los datasets

In [4]:
alumnos_delete = pd.read_pickle('../data/interim/alumnos_delete.pkl')
print('alumnos_delete shape: ',alumnos_delete.shape)

alumnos_delete shape:  (4068, 11)


In [5]:
alumnos_delete.codigo_alumno.nunique()

2673

## Datos de Alumnos

In [6]:
datdf.head()

,codigo_alumno,sexo,anio_nacimiento,anio_ingreso,codigo_carrera_ingreso
0,27,M,1974,1993,K
1,53,M,1974,1999,K
2,58,M,1974,1995,K
3,72,M,1974,1995,K
4,86,M,1974,1999,K


In [7]:
# eliminamos los alumnos de alumnos_delete
datdf = datdf[~datdf.codigo_alumno.isin(alumnos_delete.codigo_alumno)]

In [8]:
datdf.codigo_alumno.nunique()

22157

In [9]:
24830 - 22157

2673

In [10]:
datdf.to_pickle('../data/raw_cleaned/datdf_01.pkl')
datdf.to_csv('../data/raw_cleaned/datdf_01.csv',index=False)

## Datos de Cursadas

In [11]:
curdf.head()

,codigo_alumno,curso,codigo_materia,departamento,modalidad,turno,ciclo_lectivo,tipo_de_aprobacion,cantidad_recursada_regular,descripción_recursada_regular,cantidad_recursada_libre,descripción_de_recursada_libre
0,27,K5051,952037,NaN,Presencial,Noche,2006,"Regularidad, Promoción",0,NaN,0,NaN
1,27,K5052,952052,NaN,Presencial,Noche,2006,"Regularidad, Promoción",0,NaN,0,NaN
2,27,K5151,952054,NaN,Presencial,Noche,2006,"Promoción, Regularidad",0,NaN,0,NaN
3,27,K5152,952056,NaN,Presencial,Noche,2006,"Promoción, Regularidad",0,NaN,0,NaN
4,53,K4051,952046,NaN,Presencial,Noche,2010,"Regularidad, Promoción",0,NaN,0,NaN


In [12]:
# eliminamos los alumnos de alumnos_delete
curdf = curdf[~curdf.codigo_alumno.isin(alumnos_delete.codigo_alumno)]

#### criterio

Por ahora nose sabe cual es la diferencia entre `cant_recursada_regular` y `cant_recursada_libre` por lo que voy a juntar ambas variables en solo cant de resursadas y que el número se incremente segun la cantidad de registros ordenado por ciclo lectivo.

In [13]:
curdf_aux = curdf.sort_values(['codigo_alumno', 'codigo_materia', 'ciclo_lectivo'], ascending=True).copy()

# agrupar y numero de fila
curdf_aux['cant_cursada'] = curdf.groupby(['codigo_alumno', 'codigo_materia'])[['ciclo_lectivo']].transform('rank', method='first')

curdf_aux['cant_total_cursadas'] = curdf.groupby(['codigo_alumno', 'codigo_materia'])[['ciclo_lectivo']].transform('count')
curdf_aux

,codigo_alumno,curso,codigo_materia,departamento,modalidad,turno,ciclo_lectivo,tipo_de_aprobacion,cantidad_recursada_regular,descripción_recursada_regular,cantidad_recursada_libre,descripción_de_recursada_libre,cant_cursada,cant_total_cursadas
4494,10035,K4153,950310,Unidad Docente Básica Legislación y Economía,Presencial,Noche,2006,"Regularidad, Promoción",0,NaN,0,NaN,1.0,1
4497,10035,K3051,952028,NaN,Presencial,Noche,2007,"Promoción, Regularidad",0,NaN,0,NaN,1.0,1
4492,10035,K3001,952030,NaN,Presencial,Mañana,2006,"Promoción, Regularidad",0,NaN,0,NaN,1.0,1
4498,10035,K3071,952031,NaN,Presencial,Noche,2007,"Promoción, Regularidad",0,NaN,0,NaN,1.0,1
4502,10035,K4051,952033,Departamento de Ing. en Sistemas de Información,Presencial,Noche,2008,"Promoción, Regularidad",0,NaN,0,NaN,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4484,9983,K5071,952056,NaN,Presencial,Noche,2007,"Regularidad, Promoción",0,NaN,0,NaN,1.0,1
4483,9983,K5071,952063,NaN,Presencial,Noche,2007,"Promoción, Regularidad",0,NaN,0,NaN,1.0,1
4481,9983,K5051,952082,NaN,Presencial,Noche,2007,"Regularidad, Promoción",0,NaN,0,NaN,1.0,1
4485,9985,K3052,952028,NaN,Presencial,Noche,2006,"Regularidad, Promoción",0,NaN,0,NaN,1.0,1


### Limpieza de columnas




In [14]:
curdf_aux.columns

Index(['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'modalidad',
       'turno', 'ciclo_lectivo', 'tipo_de_aprobacion',
       'cantidad_recursada_regular', 'descripción_recursada_regular',
       'cantidad_recursada_libre', 'descripción_de_recursada_libre',
       'cant_cursada', 'cant_total_cursadas'],
      dtype='object')

In [15]:
COLS = ['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'turno', 'ciclo_lectivo', 'cant_cursada', 'cant_total_cursadas']

curdf_aux = curdf_aux[COLS]

In [16]:
curdf_aux.head()

,codigo_alumno,curso,codigo_materia,departamento,turno,ciclo_lectivo,cant_cursada,cant_total_cursadas
4494,10035,K4153,950310,Unidad Docente Básica Legislación y Economía,Noche,2006,1.0,1
4497,10035,K3051,952028,NaN,Noche,2007,1.0,1
4492,10035,K3001,952030,NaN,Mañana,2006,1.0,1
4498,10035,K3071,952031,NaN,Noche,2007,1.0,1
4502,10035,K4051,952033,Departamento de Ing. en Sistemas de Información,Noche,2008,1.0,1


In [17]:
curdf_aux.codigo_alumno.nunique()

22157

In [18]:
curdf = curdf_aux.copy()

In [19]:
curdf.to_pickle('../data/raw_cleaned/curdf_01.pkl')
curdf.to_csv('../data/raw_cleaned/curdf_01.csv',index=False)

## Dataset Cursadas complementario

In [20]:
curdf.codigo_materia.nunique()

705

In [21]:
findf.codigo_materia.nunique()

1099

In [22]:
# eliminamos los alumnos de alumnos_delete
findf = findf[~findf.codigo_alumno.isin(alumnos_delete.codigo_alumno)]

In [23]:
findf.codigo_materia.nunique()

1093

In [24]:
findf.codigo_alumno.nunique()

22157

In [25]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota', 'aprobado',
       'promociono', 'año_cursada', 'anio_academico_str', 'anio_cursada_str',
       'tipo_aprobacion', 'nota_comb', 'nota_num', 'nota_cual', 'nota_txt'],
      dtype='object')

In [26]:
findf.head()

,codigo_alumno,codigo_materia,anio_academico,nota,aprobado,promociono,año_cursada,anio_academico_str,anio_cursada_str,tipo_aprobacion,nota_comb,nota_num,nota_cual,nota_txt
0,27,850783,1993.0,10,10,NaN,NaN,1993.0,nan,Aprobacion,10,10.0,NaN,Aprobada
1,27,850783,1993.0,2,NaN,NaN,NaN,1993.0,nan,Aprobacion,2,2.0,NaN,No Aprobada
2,27,881347,1994.0,7,7,NaN,NaN,1994.0,nan,Aprobacion,7,7.0,NaN,Aprobada
3,27,952020,1997.0,4,4,NaN,NaN,1997.0,nan,Aprobacion,4,4.0,NaN,Aprobada
4,27,952021,1997.0,5,5,NaN,NaN,1997.0,nan,Aprobacion,5,5.0,NaN,Aprobada


In [27]:
COLS = ['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num', 'nota_cual', 'nota_txt', 'año_cursada',]

findf = findf[COLS]

findf.head()

,codigo_alumno,codigo_materia,anio_academico,nota_num,nota_cual,nota_txt,año_cursada
0,27,850783,1993.0,10.0,NaN,Aprobada,NaN
1,27,850783,1993.0,2.0,NaN,No Aprobada,NaN
2,27,881347,1994.0,7.0,NaN,Aprobada,NaN
3,27,952020,1997.0,4.0,NaN,Aprobada,NaN
4,27,952021,1997.0,5.0,NaN,Aprobada,NaN


In [28]:
findf = findf.sort_values(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num'], ascending=True).copy()

findf['cant_finales'] = findf.groupby(['codigo_alumno', 'codigo_materia'])[['anio_academico']].transform('rank', method='first')

findf.head()

,codigo_alumno,codigo_materia,anio_academico,nota_num,nota_cual,nota_txt,año_cursada,cant_finales
20181,10035,950309,2006.0,2.0,NaN,No Aprobada,NaN,1.0
20182,10035,950309,2006.0,4.0,NaN,Aprobada,NaN,2.0
20183,10035,950310,2006.0,7.0,NaN,Aprobada,2006.0,1.0
20173,10035,950611,2004.0,4.0,NaN,Aprobada,NaN,1.0
20166,10035,950701,2003.0,4.0,NaN,Aprobada,NaN,1.0


In [29]:
findf.duplicated().sum()

0

## ACA esta el problema, tiene que ser por el par alumno manteria y no solo materia

In [30]:
finaux = findf.copy()
curaux = curdf.copy()

finaux['almat'] = finaux.codigo_alumno.astype(str) + '_' + finaux.codigo_materia.astype(str)
curaux['almat'] = curaux.codigo_alumno.astype(str) + '_' + curaux.codigo_materia.astype(str)

aux = finaux[~finaux.almat.isin(curaux.almat)].copy()

aux.codigo_materia.nunique()

# eliminamos los alumnos de alumnos_delete
#findf = findf[~findf.codigo_alumno.isin(alumnos_delete.codigo_alumno)]

1034

In [31]:
aux.codigo_alumno.nunique()

16655

### Criterio

no se pueden eliminar los alumnos que tienen codigo_materia en el datset de finales pero que no esta en cursadas porque nos quedamos sin datos

- se puede tomar el dato del año de cursada como el dato faltante de cuadno se cursó

In [32]:
aux.shape

(218630, 9)

In [33]:
aux.head()


,codigo_alumno,codigo_materia,anio_academico,nota_num,nota_cual,nota_txt,año_cursada,cant_finales,almat
20181,10035,950309,2006.0,2.0,NaN,No Aprobada,NaN,1.0,10035_950309
20182,10035,950309,2006.0,4.0,NaN,Aprobada,NaN,2.0,10035_950309
20173,10035,950611,2004.0,4.0,NaN,Aprobada,NaN,1.0,10035_950611
20166,10035,950701,2003.0,4.0,NaN,Aprobada,NaN,1.0,10035_950701
20167,10035,950702,2003.0,2.0,NaN,No Aprobada,NaN,1.0,10035_950702


In [34]:
# criterio
# si anio cursada no tiene datos, completar con el mismo año que rindio final que esta en anio_academico
aux['anio_cursada'] = aux['año_cursada'].combine_first(aux.anio_academico)
aux.head()

,codigo_alumno,codigo_materia,anio_academico,nota_num,nota_cual,nota_txt,año_cursada,cant_finales,almat,anio_cursada
20181,10035,950309,2006.0,2.0,NaN,No Aprobada,NaN,1.0,10035_950309,2006.0
20182,10035,950309,2006.0,4.0,NaN,Aprobada,NaN,2.0,10035_950309,2006.0
20173,10035,950611,2004.0,4.0,NaN,Aprobada,NaN,1.0,10035_950611,2004.0
20166,10035,950701,2003.0,4.0,NaN,Aprobada,NaN,1.0,10035_950701,2003.0
20167,10035,950702,2003.0,2.0,NaN,No Aprobada,NaN,1.0,10035_950702,2003.0


In [35]:
# me quedo con los datos para agregarlos al dataset de cursadas

COLS = ['codigo_alumno', 'codigo_materia', 'anio_cursada']
cursadas_criterio = aux[COLS].copy().drop_duplicates()
cursadas_criterio.shape

(192407, 3)

In [37]:
cursadas_criterio.head()

,codigo_alumno,codigo_materia,anio_cursada
20181,10035,950309,2006.0
20173,10035,950611,2004.0
20166,10035,950701,2003.0
20167,10035,950702,2003.0
20176,10035,950703,2005.0


In [38]:
# este join no deberia tener interseccion
extra = pd.merge(cursadas_criterio, curdf, on=['codigo_alumno', 'codigo_materia'], how='inner')
extra.shape

(0, 9)

In [39]:
curdf.columns

Index(['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'turno',
       'ciclo_lectivo', 'cant_cursada', 'cant_total_cursadas'],
      dtype='object')

In [40]:
cursadas_criterio.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_cursada'], dtype='object')

In [41]:
cursadas_criterio.columns = ['codigo_alumno', 'codigo_materia', 'ciclo_lectivo']

In [42]:
# completo el resto de las variables para dsps tener el dataset para integrarloa cursadas

cursadas_criterio['curso'] = 'S/D'
cursadas_criterio['departamento'] = 'S/D' # este se puede completar dsps con los datos de organigrama
cursadas_criterio['turno'] = 'S/D'
cursadas_criterio['cant_cursada'] = 1
cursadas_criterio['cant_total_cursadas'] = 1

cursadas_criterio = cursadas_criterio[curdf.columns]


In [43]:
cursadas_criterio.shape

(192407, 8)

In [44]:
cursadas_criterio.columns

Index(['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'turno',
       'ciclo_lectivo', 'cant_cursada', 'cant_total_cursadas'],
      dtype='object')

In [45]:
# dataset de cursadas complementarias al original por datos obtenidos del dataset de Finales
cursadas_criterio.to_pickle('../data/raw_cleaned/curdf_complementario_01.pkl')
cursadas_criterio.to_csv('../data/raw_cleaned/curdf_complementario_01.csv', index=False)

## Dataset de Finales

In [46]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num',
       'nota_cual', 'nota_txt', 'año_cursada', 'cant_finales'],
      dtype='object')

In [47]:
findf = findf[['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num', 'nota_cual', 'nota_txt', 'cant_finales']]

In [48]:
findf.to_pickle('../data/raw_cleaned/findf_01.pkl')
findf.to_csv('../data/raw_cleaned/findf_01.csv', index=False)